# **Assignment: Adversarial search strategies and Decision Trees**

## **1. Connect 4 Game**

Iniciamos este trabalho criando uma implementação de uma classe que representa o jogo Connect 4.

In [ ]:
class FourGame():
    # Parameters | self: Class FourGame instance | columns: Integer number of columns for the game | lines: Integer number of lines for the game
    def __init__(self, columns, lines):  # Class constructor (there is no need to declare the attributes since the self.attribute does it for us)
        self.state = [['-' for _ in range(columns)] for _ in range(lines)]
        self.columns = columns
        self.lines = lines
        self.plays = 0
        self.toPlay = 'O'
        self.result = None


    # Parameters | self: Class FourGame instance
    def __str__(self):  # Returns the string who graphically represents the matrix of the game
        str = "=============================\n"
        for line in self.state:  # Iterate through the columns
            for sign in line:  # Iterate through the lines
                str += '| ' + sign + ' '
            str += "|\n"
        str += "=============================\n"
        str += "  1   2   3   4   5   6   7  \n"
        return str


    # Parameters | self: Class FourGame instance | column: Integer number of column to play | Character symbol ('X', 'O')
    def __insertSymbol(self, column, symbol):  # Private method to insert a symbol into the array
        for i in range(self.lines-1, -1, -1):
            if self.state[i][column] == '-':
                self.state[i][column] = symbol
                if symbol == 'X':
                    self.toPlay = 'O'
                else:
                    self.toPlay = 'X'
                return i
        return -1  # Returns in case the column is full

    def __checkRepetitions(self, line, column, lineCount, columnCount, symbol):
        count = 0
        lineI = line
        columnI = column
        for _ in range(2):

            while lineI <= 5 and lineI >= 0 and columnI <= 6 and columnI >= 0 and self.state[lineI][columnI] == symbol:
                count += 1
                columnI += columnCount
                lineI += lineCount
                if count == 4: 
                    self.result = symbol
                    return True
 
            columnCount *= -1
            lineCount *= -1
            columnI = column + columnCount
            lineI = line + lineCount

        return False
    
    def __checkWin(self, column, line, symbol):

        # Check for win vertically
        if self.__checkRepetitions(line, column, -1, 0, symbol):
            return True
            
        # Check for win horizontallys
        if self.__checkRepetitions(line, column, 0, 1, symbol):
            return True
        
        # Check for win main diagonal
        if self.__checkRepetitions(line, column, -1, 1, symbol):
            return True

        # Check for win inverse diagonal
        if self.__checkRepetitions(line, column, 1, 1, symbol):
            return True
        
        return False
        
    
    # Parameters | self: Class FourGame instance | column: Integer number of column to play | Character symbol ('X', 'O')
    # Return: string or false
    def makeMove(self, column, symbol):
        
        line = self.__insertSymbol(column, symbol)
        if line == -1: return -1, ''  # Invalid move, the column is full

        self.plays += 1
        
        # Verify if it should end the game (in case if someone wins or the board is full)
        if self.__checkWin(column, line, symbol): 
            return 2, symbol
        elif (self.plays == self.columns*self.lines): 
            return 1, ''
        return 0, ''

    def gameOver(self):
        return self.result is not None or self.plays == self.columns * self.lines
    
    def gameDraw(self):
        return self.plays == self.columns * self.lines
    
    def getLegalMoves(self):
        legalMoves = []
        for i in range(7):
            if self.state[0][i] == '-':
                legalMoves.append(i)

        return legalMoves

### **Estruturas de Dados**

Implementámos três estruturas de dados distintas para dar suporte à lógica do jogo e aos algoritmos. Node, Vector e Queue.

#### **Node**

In [1]:
class Node():

    def __init__(self, move, state, parent):
        self.move = move
        self.state = state
        self.parent = parent
        self.children = []
        self.N = 0
        self.Q = 0
    
    def setPathCost(self, cost):
        self.pathCost = cost

    def getPathCost(self):
        return self.pathCost
    
    def setChildren(self, children):
        self.children = children

**Atributos**

Move -> Coordenadas do movimento feito a ser representado pelo nó. Vetor de duas dimensões com a posição da coluna e linha

State -> Estado atual do tabuleiro do jogo. Matriz de caracteres que representam os símbolos do jogo.

Parent -> Nó pai do nó representado. Referência para um nó.

Children -> Lista dos nós filhos do nó. Lista de referências para nós.

pathCost -> Custo do caminho + heurística para passar para nó

**Métodos**

setPathCost -> Atribui um valor ao atributo pathCost

getPathCost -> Retorna o valor do atributo pathCost

setChildren -> Atribui um valor ao atributo children

#### **Vector**

In [ ]:
class Vector():

    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def setX(self, x):
        self.x = x

    def setY(self, y):
        self.y = y
    
    def getX(self):
        return self.x

    def getY(self):
        return self.y

**Atributos**

X -> Valor X do vetor (int)

Y -> Valor Y do vetor (int)

**Métodos**

setX -> Atribui um valor ao atributo X

setY -> Atribui um valor ao atributo Y 

getX -> Retorna o valor do atributo X

getY -> Retorna o valor do atributo Y 

#### **Queue**

In [ ]:
class Queue():

    def __init__(self):
        self.stack = []
        self.size = 0

    def isEmpty(self):
        if self.size ==0:
            return True
        return False

    def pop(self):
        if self.size != 0:
            self.size = self.size - 1
            return self.stack.pop(self.size)
        else:
            return None

    def top(self):
        if self.size != 0:
            return self.stack.pop(self.size-1)
        else:
            return None
        
    def add(self, value):
        self.stack.append(value)
        self.size = self.size + 1

**Atributos**

Stack -> Lista para armazenar os valores da fila

Size -> Tamanho da fila

**Métodos**

isEmpty -> Verifica se a lista está ou não vazia

pop -> Remove e retorna o elemento do topo da fila

top -> Apenas retorna o elemento do topo da fila

add -> Adiciona um elemento no topo da fila

### **Heurística**

A heurística inicialmente utilizada demonstrou ser eficaz de forma geral, embora por vezes não dê prioridade às jogadas que levem a uma vitória mais rápida. Ainda assim, optámos por explorar uma heurística alternativa com o objetivo de realizar comparações.

A primeira abordagem segue a seguinte fórmula:
**Valor = ValorPeçasIA - ValorPeçasDoAdversário**

Este valor é obtido ao percorrer todas as sequências possíveis de quatro casas no tabuleiro e atribuir pontuações específicas, conforme a quantidade de peças consecutivas de um único jogador:

- 1 peça -> 1 ponto
- 2 peças -> 10 pontos
- 3 peças -> 50 pontos
- 4 peças (vitória) -> 512 pontos

**Nota:** No entanto, se uma sequência contiver peças de ambos os jogadores, é considerada inválida para a vitória, pelo que é atribuída a pontuação de zero pontos.

Posteriormente, testámos uma variação com pesos diferentes (0.1, 0.3, 0.9, 3.2), baseada num relatório académico externo. Contudo, esta versão revelou-se menos eficiente, permitindo empates frente ao algoritmo MinMax, especialmente com uma profundidade de apenas 3 níveis.

Outra heurística que experimentamos baseia-se na valorização das posições do tabuleiro: casas centrais recebem maior pontuação, como ilustrado num esquema habitualmente usado em implementações com Monte Carlo Tree Search. 

<div style="text-align: center;">
    <img src="esquemas/heuristica.png" alt="Esquema MCTS" width="320" class="center"/>
</div>

Isoladamente, esta abordagem apresentou bons resultados em fases iniciais do jogo, mas revelou-se insuficiente em situações mais complexas, levando a decisões incorretas.

Porém, ao combinar esta valorização posicional com a heurística original, conseguimos uma melhoria notável - especialmente no contexto do MinMax. Já no caso do algoritmo A*, esta adição mostrou ter pouco impacto, uma vez que o A* tende a falhar por não considerar adequadamente as jogadas do adversário.

In [ ]:
# Regras de valoração de combinações do segmento (segmentos de 4 slots)
# Parâmetros:

# segment | type: list de caracteres de tamanho 4   -> representa um segmento de 4 slots com os símbolos de cada slot
# IaSymbol | type: string                           -> representa o símbolo que IA está a usar para jogar

# returns: type: int                                -> Retorna a valoração correta para a combinação de número de símbolos X e O encontrados no segmento segundo as regras dadas para calculo da heurística

def heuristicVal(segment, IaSymbol):
    
    # Define qual símbolo é usado pelo humano com base no dado para a IA
    if IaSymbol == 'X':
        HumanSymbol = 'O'
    else:
        HumanSymbol = 'X'

    # Conta o número
    IA_count = segment.count(IaSymbol)
    Human_count = segment.count(HumanSymbol)

    if Human_count == 4 and IA_count == 0:
        return -512                                                     # Valor para 4 Símbolos do humano e 0 Símbolos  da IA
    elif Human_count == 3 and IA_count == 0:
        return -50                                                      # Valor para 3 Símbolos do humano e 0 Símbolos  da IA
    elif Human_count == 2 and IA_count == 0:
        return -10                                                      # Valor para 2 Símbolos do humano e 0 Símbolos  da IA
    elif Human_count == 1 and IA_count == 0:
        return -1                                                       # Valor para 1 Símbolos do humano e 0 Símbolos  da IA
    elif Human_count == 0 and IA_count == 0:
        return 0                                                        # Sem O ou X
    elif IA_count == 1 and Human_count == 0:
        return 1                                                        # Valor para 1  Símbolos da IA e 0 Símbolos do humano
    elif IA_count == 2 and Human_count == 0:
        return 10                                                       # Valor para 2  Símbolos da IA e 0 Símbolos do humano
    elif IA_count == 3 and Human_count == 0:
        return 50                                                       # Valor para 3  Símbolos da IA e 0 Símbolos do humano
    elif IA_count == 4 and Human_count == 0:
        return 512                                                      # Valor para 4  Símbolos da IA e 0 Símbolos do humano
    else:
        return 0                                                        #Combinação de O e X | nenhum padrão encontrado


# Cálculo da heurística através da geração dos segmentos e soma dos seus valores 
# Parâmetros: 

# boardState | matrix (list of lists): matrix de caracteres             -> representa o estado atual do tabuleiro do jogo
# IaSymbol | type: string                                               -> representa o símbolo que AI está a usar para jogar

# returns: type: int                                                    -> Retorna a heurística do boardState dado

def heuristicCalculate(boardState, IaSymbol, coords):

    ValueGrade = [[3,4,5,7,5,4,3],[4,6,8,10,8,6,4],[5,8,11,13,11,8,5],[5,8,11,13,11,8,5],[4,6,8,10,8,6,4],[3,4,5,7,5,4,3]]

    heuristic = 0

    if coords:
        heuristic =  ValueGrade[coords.y][coords.x]

    # Segmentos horizontais
    for row in boardState:
        for i in range(len(row) - 3):
            segment = row[i:i+4]                                        # Segmento a ser valorado
            heuristic += heuristicVal(segment, IaSymbol)

    # Segmentos verticais
    for j in range(len(boardState[0])):
        for i in range(len(boardState) - 3):
            segment = [boardState[i+k][j] for k in range(4)]            # Segmento a ser valorado
            heuristic += heuristicVal(segment, IaSymbol)

    # Diagonal segments (superior-esquerdo to inferior-direito)
    for i in range(len(boardState) - 3):
        for j in range(len(boardState[0]) - 3):
            segment = [boardState[i+k][j+k] for k in range(4)]          # Segmento a ser valorado
            heuristic += heuristicVal(segment, IaSymbol)

    # Segmentos das Diagonais (superior-direito até inferior-esquerdo)
    for i in range(len(boardState) - 3):
        for j in range(3, len(boardState[0])):
            segment = [boardState[i+k][j-k] for k in range(4)]          # Segmento a ser valorado
            heuristic += heuristicVal(segment, IaSymbol)

    # Bónus de movimento
    move_bonus = 16 if IaSymbol == 'X' else -16                         # Define o bónus dependendo de quem joga
    heuristic += move_bonus                                             # Adiciona o bónus

    return heuristic

### **Algoritmo A***

**Implementação:**

In [ ]:
from Game.dataStructs.myQueue import Queue
from Game.dataStructs.node import Node
from Game.dataStructs.vector import Vector
from Game.searchAlgos.heuristic import heuristicCalculate

# Máximo entre dois inteiros
# Parâmetros: 

# a, b | type: int                                       -> Inteiros a ser comparados

# returns: type: int                                     -> Retorna o máximo entre a e b

def max(a, b):
    if a > b:
        return a
    else:
        return b

# Classe que representa o algoritmo Astar                                                                                                                                                  
# Atributos:                                                                                                                                                                                
       
# frontier | type: Queue instance                        -> Representa a fronteira ou seja os próximos caminhos a partir do estado dado
# symbol | type: string                                  -> Representa o símbolo a ser jogado pela IA
# monotony | type: int/float                             -> Representa o valor da heurística (custo do caminho é irrelevante neste jogo) do ultimo estado escolhido

# Métodos:

# __bestMove | privado                                   -> Retorna a coluna do move do melhor nó ou seja o nó com heurística mais alta no caso
# __setFrontier | privado                                -> Define o atributo frontier com os nós filhos (neste caso não adiciona os nós anteriormente analizados pois não podemos voltar para trás neste jogo)
# play | publico                                         -> Retorna a jogada a fazer pela IA

class Astar():

    # Construtor da classe
    # Parâmetros: 
    
    # self | type: int                                    -> Referência ao objeto Astar usado 
    # symbol | type: string                               -> Símbolo jogado pela IA a ser guardado como argumento da classe
    
    # A monotonia é ainda inicializada com o -inf para que possa ser ignorada na primeira jogada da IA

    def __init__(self, symbol):
        self.frontier = Queue()
        self.symbol = symbol
        self.monotony = float('-inf')
    
    # Best Move
    # Parâmetros: 
    
    # self | type: int                                                  -> Referência ao objeto Astar usado 
    
    # return: type: int                                                 -> Retorna a coluna do move do melhor nó ou seja o nó com heurística mais alta no caso

    def __bestMove(self):

        if self.frontier.isEmpty():                                     # Verifica se exitem jogadas possíveis (nós filhos)
            return False

        bestMoveNode = self.frontier.pop()                              # Escolhe o melhor nó de entre os nós filhos (ou seja aquele com melhor heurística)
        while ((newNode := self.frontier.pop()) != None):
            if newNode.pathCost > bestMoveNode.pathCost:
                bestMoveNode = newNode
        
        self.monotony = bestMoveNode.pathCost                           # Atualiza a monotonia para a proxima jogada da IA

        return bestMoveNode.move.getX()
    
    # setFrontier
    # Parâmetros: 
    
    # self | type: int                                                  -> Referência ao objeto Astar usado 
    # actualState | type: matrix (list of lists): matrix de caracteres  -> Representa o estado atual do tabuleiro do jogo

    # Define o atributo frontier com os nós filhos (neste caso não adiciona os nós anteriormente analizados pois não podemos voltar para trás neste jogo)
    def __setFrontier(self, actualState):
        for column in range(0,7):
            for line in range(5,-1,-1):
                if actualState[line][column] == '-':

                    newState = list(map(list, actualState))             # Cria uma cópia do tabuleiro numa referência de memória diferente
                    newState[line][column] = self.symbol                # Atualiza o tabuleiro com a jogada representada por o nó a ser calculado

                    node = Node(Vector(column, line), newState, None)   # Cria o nó com o move e o estado do jogo que representa

                    # Na linha abaixo deveria ser somado o custo do caminho ao heuristicCalculate mas dado que neste jogo o custo é irrelevante não foi feito.
                    node.setPathCost(max(self.monotony, heuristicCalculate(node.state, self.symbol, node.move))) # Calcula e guarda a heurística do nó especifico (o custo do caminho é irrelevante neste jogo)

                    self.frontier.add(node)                             # Adiciona o nó à fronteira
                    break

    # play
    # Parâmetros: 
    
    # self | type: int                                                  -> Referência ao objeto Astar usado 
    # game | type: matrix (list of lists): matrix de caracteres         -> Representa o estado atual do tabuleiro do jogo
    
    # return | type: int                                                -> Retorna a jogada a fazer pela IA

    def play(self, game, _):
        self.__setFrontier(game.state)                                  # Define a fronteira
        
        return self.__bestMove()                                        # Verifica e retorna o melhor dos nós da fronteira

**Dificuldades encontradas e aspetos considerados durante a implementação do algoritmo A***

Na fase inicial da implementação, rapidamente percebemos que ao contrário de uma aplicação tradicional do algoritmo A*, o nosso caso específico apresentava características que dispensavam certas funcionalidades típicas, como a necessidade de retroceder no caminho percorrido. Isto deve-se ao facto de, no jogo em questão, as peças jogadas permanecerem fixas no tabuleiro, não havendo possibilidade de remoção.

Outra observação importante foi a irrelevância do custo associado à jogada em si, uma vez que todas as colunas representam, do ponto de vista do algoritmo, um custo idêntico. Como consequência, concluímos que a nossa implementação se aproximava mais de uma abordagem Greedy do que de um verdadeiro A*, já que estávamos a considerar apenas a heurística sem integrar um custo de caminho acumulado.

A principal questão foi o porquê de o algoritmo ignorar completamente as jogadas do oponente. Após análise, percebemos que devia-se ao facto de ser de o algoritmo ser de natureza não adversarial, o que tornava ineficaz em cenários competitivos.

**Limitações do algoritmo no contexto do jogo**

Durante o desenvolvimento e respetivos testes com o algoritmo A*, identificamos dois problemas principais:

O primeiro diz respeito à sua incapacidade de considerar os movimentos do adversário. Por ser não adversarial, o algoritmo não antecipa jogadas do oponente, o que torna vulnerável a perdas fáceis, sobretudo quando o humano é o primeiro a jogar.

O segundo ponto não apresenta um erro, mas sim uma característica do problema. Como jogar numa coluna ou noutra tem sempre o mesmo custo, o algoritmo acaba por ignorar essa componente do A*, focando-se apenas na heurística. Na prática, isso faz com que o comportamento seja muito semelhante ao de um algoritmo Greedy.

Tendo isto em consideração, optamos por simplificar a implementação, utilizando apenas a heurística para a decisão das jogadas. Embora o comportamento seja praticamente igual ao de um algoritmo Greedy, garantimos que a heurística usada é estável e consistente, para manter alguma semelhança ao A*.

### **Algoritmo MinMax**

**Implementação:**

In [ ]:
from Game.dataStructs.vector import Vector
from Game.dataStructs.node import Node
from Game.searchAlgos.heuristic import heuristicCalculate

# Atributos:                                                                                                                                                                                
     
# MaxSymbol | type: character       -> Representa o simbolo usado nos máximos (Simbolo da IA)
# MinSymbol | type: character       -> Representa o símbolo usado nos mínimos

# Métodos:
#
#  __gameAlreadyWon                 ->  Retorna a coluna do move do melhor nó ou seja o nó com heurística mais alta no caso
#  __minimax                        ->  Função recursiva para determinar o máximo e minimo dos nós alternadamente (implementação do minMax em si)
#  __getChildren                    ->  Define os nós filhos do nó a ser analisado e dá calcula a heurística deles e o estado que representam
#  play                             ->  Retorna a jogada a fazer pela IA
#
class MinMax():

    # Construtor da classe
    # Parâmetros: 
    
    # self      | type: int         -> Referência ao objeto MinMax usado
    # MaxSymbol | type: character   -> Representa o simbolo usado nos máximos (Simbolo da IA)
    # MinSymbol | type: character   -> Representa o símbolo usado nos mínimos
    
    def __init__(self, MaxSymbol, MinSymbol):
        self.MaxSymbol = MaxSymbol
        self.MinSymbol = MinSymbol
    
    # gameAlreadyWon
    # Parâmetros:
    
    # self | type: int                                                      -> Referência ao objeto MinMax usado
    # gameState | type: matrix (list of lists): matrix de caracteres        -> Representa o estado do jogo neste ponto
    
    # return: type: int                                                     -> Retorna a heurística em caso de vitória, empate ou derrota ou None

    def __gameAlreadyWon(self, gameState):
        heuristic = heuristicCalculate(gameState, self.MaxSymbol, None)     # Calcula a heurística do estado neste ponto
        if heuristic >= 512 or heuristic <= -512 or heuristic == 0:         # Verifica se é vitória, empate ou derrota
            return heuristic
        return None

    # minimax
    # Parâmetros: 
    
    # self | type: int                                                      -> Referência ao objeto MinMax usado
    # children | type: list of nodes                                        -> Lista com os nós filhos do nó "expandido"
    # depth | type: int                                                     -> Altura restante a analisar
    # maximizingPlayer | type: bool                                         -> Se é ou não uma altura de comparar máximos ou não (se não for são mínimos)
    # actualGame | type: matrix (list of lists): matrix de caracteres       -> Representa o estado do jogo neste ponto
    
    # return: type: int                                                     ->  Retorna o valor máximo ou minimo dependendo da fase e o melhor nó


    def __minimax(self, children, depth, maximizingPlayer, actualGame):

        if depth == 0:                                                       # Verifica se a já verificamos a uma altura suficiente (defenida posteriormente)
            return 0, None
 
        # Primeiro expande até à profundidade pretendida e posteriormente ao voltar para trás escolhe o nó maximo ou minimo dependendo do "turno" (forma alternada começando sempre no Max nesta implementação)
        if maximizingPlayer:
            heuristicVal = self.__gameAlreadyWon(actualGame)                # Verifica se houve ou não o fim do jogo e retorna uma heurística caso aconteça
            if heuristicVal and (heuristicVal >= 512 or heuristicVal == 0):
                return heuristicVal, None                                   # Retornando tal heurística como valor máximo em caso de vitória ou empate
 
            max_eval = float('-inf')                                        # Inicializa o valor máximo como -inf para a primeira comparação
            best_move = None                                                # Inicializa o bestMove
            for child in children:                                          # Percorre os nós filhos do nó atual a ser analizado

                eval, bestChild = self.__minimax(self.__getChildren(child.state, self.MinSymbol), depth - 1, False, child.state) # Para cada nó filho aplica o minmax mas agora para a busca de minimos

                if bestChild == None:                                       # Caso o nó maximo dos nós seja vitória ou empate o eval recebe o valor da heurística do nó filho a ser analizado
                    eval = child.getPathCost()

                if eval > max_eval:                                         # Verifica se o nó é maximo comparado aos já analizado
                    max_eval = eval
                    best_move = child

            return max_eval, best_move                                      # Retorna o nó maximo e o seu valor
        else:
            heuristicVal = self.__gameAlreadyWon(actualGame)                # Verifica se houve ou não o fim do jogo e retorna uma heurística caso aconteça
            if heuristicVal and (heuristicVal <= -512 or heuristicVal == 0): 
                return heuristicVal, None                                   # Retornando tal heurística como valor mínimo em caso de vitória ou empate

            min_eval = float('inf')                                         # Inicializa o valor mínimo como inf para a primeira comparação
            best_move = None                                                # Inicializa o bestMove
            for child in children:                                          # Percorre os nós filhos do nó atual a ser analizado

                eval, bestChild = self.__minimax(self.__getChildren(child.state, self.MaxSymbol), depth - 1, True, child.state) # Para cada nó filho aplica o minmax mas agora para a busca de máximos
                
                if bestChild == None:                                       # Caso o nó mínimo dos nós seja vitória ou empate o eval recebe o valor da heurística do nó filho a ser analizado
                    eval = child.getPathCost()
                
                if eval < min_eval:                                         # Verifica se o nó é maximo comparado aos já analizado
                    min_eval = eval
                    best_move = child

            return min_eval, best_move                                      # Retorna o nó mínimo e o seu valor

    # getChildren
    # Parâmetros: 
    
    # self | type: int                                                      -> Referência ao objeto MinMax usado
    # game | type: matrix (list of lists): matrix de caracteres             -> Representa o estado do jogo neste ponto
    # symbolToPlay | type: character                                        -> Simbolo a jogar no momento
    
    # return: type: list of nodes                                           -> Retorna os nós filhos do estado passado

    def __getChildren(self, game, symbolToPlay):
        children = []
        for column in range(0,7):
            for line in range(5, -1, -1):
                if game[line][column] == '-':                                               # Procura pela linha disponível na coluna caso exista
                    
                    newGame = list(map(list, game))                                         # Copia o estado do tabuleiro para uma referência de memória diferente
                    newGame[line][column] = symbolToPlay                                    # Atualiza esse estado com o simbolo no move a jogar
                    node = Node(Vector(column, line), newGame, None)                        # Cria um nó para esse move e estado
                    
                    node.setPathCost(heuristicCalculate(game, self.MaxSymbol, node.move))   # Calcula a heurística para esse estado
                    children.append(node)                                                   # adiciona esse nó á lista dos filhos
                    break
        return children

    # play
    # Parâmetros: 
    
    # self | type: int                                                      -> Referência ao objeto MinMax usado
    # game | type: matrix (list of lists): matrix de caracteres             -> Representa o estado do jogo neste ponto
    
    # return: type: int                                                     -> Retorna a coluna da melhor jogada

    def play(self, game, _):
        newGame = list(map(list, game.state))                               # Copia o estado do jogo para outro endereço de memória
        frontier = self.__getChildren(newGame, self.MaxSymbol)              # Busca os nós filhos do estado atual
    
        _, melhor = self.__minimax(frontier, 3, True, newGame)              # Aplica o minmax começando pelo máximo

        return melhor.move.getX()                                           # Retorna a coluna da melhor jogada

**Análise do Minimax**

O algortimo MiniMax opera de forma recursiva, analisando todas as possíveis jogadas a partir de uma dada configuração do tabuleiro.

A cada nível da recursão, utiliza uma função heurística para avaliar os nós, com base nas jogadas válidas obtidas através da função setFrontier(). O algortimo escolhe a jogada que maximiza essa avaliação quando está no turno do jogador principal.

Durante o jogo contra um humano, o algoritmo alterna entre maximizar a sua própria heurística e minimizar a do adversário, simulando ambos os lados da partida. Esta alternância permite que o MiniMax tome descisões estratégicas, antecipando possíveis jogadas do oponente. Também é capaz de jogar contra si próprio, aplicando o mesmo princípio recursivo.

**Esquema:**

<div style="text-align: center;">
    <img src="esquemas/minimax.png" alt="Esquema MCTS" width="1200" class="center"/>
</div>

**Vantagens em relação ao A\* e limitações**

A principal motivação para implementação do MiniMax foi testar a eficácia da heurística num contexto adversarial - algo mais adequado à natureza deste jogo.

Comparando com o A*`, a diferença é clara: o MiniMax considera também as ações do adversário. Em vez de jogar apenas para ganhar, o algoritmo aprende a defender-se, bloqueando jogadas perigosas e evitando derrotas.

Verificamos, no entanto, que o MiniMax nem sempre escohle o caminho mais curto para a vitória, o que se deve às limitações da heurística utilizada, que nem sempre favorece finais rápidos.

Quanto à performance, esta depende diretamente da profundidade de análise. Uma profundidade de 3 revelou-se um bom compromisso entre tempo de execução e eficácia: o algoritmo tornou-se bastante difícil de vencer (de facto, não o conseguimos vencer), mantendo uma performance aceitável. Profundidades superiores aumentam ligeiramente o tempo de resposta, mas tornam o algoritmo ainda mais robusto.

### **Algoritmo Monte Carlo Tree Search**

##### **Implementação:**

In [ ]:
from Game.dataStructs.node import Node
from copy import deepcopy
import random
import time

class MCTS():
    def __init__(self, iaSymbol, game):
        self.root = Node(None, None, None)                                                      # Inicia a raiz R com valores None
        self.rootState = deepcopy(game)                                                         # Define o estado do jogo na raiz como uma cópia do objeto game da classe FourGame
        self.symbol = iaSymbol                                                                  # Define o símbolo do MCTS como o iaSymbol (argumento passado na interface)
        self.playerSymbol = (iaSymbol == 'X' and 'O') or (iaSymbol == 'O' and 'X')              # Define o símbolo do jogador como o símbolo contrário do MCTS
        self.iaSymbol = iaSymbol                                                                
        self.numRollouts = 0                                                                    # Inicializa o número de Rollouts a 0 (dado estatístico)
        self.runTime = 0                                                                        # Inicializa o tempo decorrido a 0 (dado estatístico)

    def __rotateSymbol(self):                                                                   # Uma função para definir a rotação dos símbolos
        if self.symbol == 'X':
            self.symbol = 'O'
            return 'X'
        else:
            self.symbol = 'X'
            return 'O'

    def __selection(self):
        node = self.root
        state = deepcopy(self.rootState)

        while len(node.children) != 0:
            children = node.children.values()
            max_value = max(children, key=lambda n: n.value()).value()
            maxChildren = [n for n in children if n.value() == max_value]

            node = random.choice(maxChildren)
            state.makeMove(node.move+1, self.__rotateSymbol())

            if node.N == 0:
                return node, state

        if self.__expand(node, state):
            node = random.choice(list(node.children.values()))
            state.makeMove(node.move+1, self.__rotateSymbol())
            
        return node, state
    
    def __expand(self, parent, state):

        if state.gameOver():
            return False
        
        children = [Node(move, None, parent) for move in state.getLegalMoves()]
        parent.setChildren(children)

        return True

    def rollOut(self, state):
        while not state.gameOver():
            state.makeMove(random.choice(state.getLegalMoves())+1, self.__rotateSymbol())
        
        if state.gameDraw():
            return ''
        
        return state.result
    
    def backPropagation(self, node, turn, outcome):

        reward = 0 if outcome == turn else 1

        while node is not None:
            node.N += 1
            node.Q += reward
            node = node.parent
            if outcome == '':
                reward = 0
            else:
                reward = 1 - reward

    def search(self, timeLimit):
        startTime = time.process_time()

        numRollouts = 0

        while time.process_time() - startTime < timeLimit:
            node, state = self.__selection()
            outcome = self.rollOut(state)
            self.backPropagation(node, self.symbol, outcome)
            numRollouts += 1

        self.runTime = time.process_time() - startTime
        self.numRollouts = numRollouts

    def bestMove(self):

        max_value = max(self.root.children.values(), key=lambda n: n.N).N
        max_nodes = [n for n in self.root.children.values() if n.N == max_value]
        bestChild = random.choice(max_nodes)

        return bestChild
    
    def moveRoot(self, move, symbol):
        if move in self.root.children:                                                          # Se a jogada estiver nos filhos do nó que representa a raiz:
            self.rootState.makeMove(move+1, symbol)                                             # Realizamos a jogada em questão.
            self.root = self.root.children[move]                                                # Atualizamos a raiz para descer com a jogada em questão.
            return

        if move:                                                                                # No caso de ser a primeira jogada:
            self.rootState.makeMove(move+1, symbol)                                             # Realizamos a jogada em questão.
        self.root = Node(None, None, None)                                                      # Definimos a root como um nó vazio.


    def play(self, _, move):

        self.moveRoot((move and move-1), self.playerSymbol)                                     # Definimos a root com o move do jogador.

        self.search(10)                                                                         # Fazemos uma execução do algoritmo com tempo limite de 16 segundos.
        
        mcts_move = self.bestMove().move                                                        # Determinamos a melhor jogada possível com o MCTS.
        
        self.moveRoot(mcts_move, self.iaSymbol)                                                 # Atualizamos a raiz, tendo em conta essa mesma jogada escolhida pelo MCTS.

        self.symbol = self.iaSymbol                                                             # Definimos o símbolo corretamente.
        
        return mcts_move                                                                        # Por fim, retornamos a melhor jogada possível à interface.

##### **Análise MCTS**

O Monte Carlo Tree Search é um algoritmo de pesquisa avançado, amplamente utilizado em jogos complexos como Go e Xadrez, pela sua capacidade de lidar eficientemente com grandes espaços de estados.

Ao contrário de abordagens tradicionais, o MCTS não explora a totalidade da árvore de jogo. Em vez diso, realiza várias simulações aleatórias e avalia quais os ramos mais promissores com base nos resultados obtidos. Cada jogada é avaliada tendo em conta o número de vitórias associadas, o que permite melhorar progressivamente a qualidade das decisões.

O funcionamento do MCTS divide-se em quatro fases fundamentais que iremos apresentar de seguida.

##### **Esquema:**

<div style="text-align: center;">
    <img src="esquemas/MCTS.png" alt="Esquema MCTS" width="1200" class="center"/>
</div>

#### **Selection:**

In [ ]:
def __selection(self):
        node = self.root                                                                        # Definimos o nó que iremos manipular através da root atual.
        state = deepcopy(self.rootState)                                                        # Fazemos uma deepcopy do estado do jogo na raiz, para que não seja alterado o jogo em vigor.

        while len(node.children) != 0:                                                          # Enquanto o nó tiver filhos:
            children = node.children.values()                                                   # Obtemos os nós através do dicionário (este é do tipo dict_values()).
            max_value = max(children, key=lambda n: n.value()).value()                          # Entre esses nós, descobrimos o que tem o maior valor segundo a fórmula UCT (definida na classe Node).
            maxChildren = [n for n in children if n.value() == max_value]                       # Comparamos o maior valor com os restantes nós, porque podem existir vários com o mesmo valor.

            node = random.choice(maxChildren)                                                   # Na lista de nós com os maiores valores, escolhemos de forma aleatória um (para que seja imparcial).
            state.makeMove(node.move+1, self.__rotateSymbol())                                  # Na cópia que fizemos do jogo, fazemos a jogada correspondente ao nó que obtemos.

            if node.N == 0:                                                                     # Se o nó ainda não foi explorado:
                return node, state                                                              # Retornamos esse mesmo nó e o respetivo estado de jogo após a jogada que correspondia a esse nó.

        if self.__expand(node, state):                                                          # Se for possível fazer a expansão:
            node = random.choice(list(node.children.values()))                                  # Escolhemos aleatóriamente um dos nós.
            state.makeMove(node.move+1, self.__rotateSymbol())                                  # Fazemos a respetiva jogada, indicada por esse nó.
            
        return node, state                                                                      # No fim, retornamos o nó e o estado do jogo após a jogada.

Na *Selection*, começamos a partir da raiz R, e selecionamos nós filhos sucessivamente até atingirmos um nó folha L. A raiz é o estado atual do jogo e uma folha é qualquer nó que não tenha filhos mas que tenha um potencial filho onde nenhum/a *Rollout/Simulation* tenha sido iniciado. Quando chegarmos a esse nó folha iremos fazer a *Expansion*.



##### **Expansion:**

In [ ]:
def __expand(self, parent, state):
    if state.gameOver():                                                                        # Se o jogo estiver num estado final:
        return False                                                                            # Retornamos False, indicando que não deve ser feita a expansão (dado que esta seria impossível).
        
    children = [Node(move, None, parent) for move in state.getLegalMoves()]                     # Caso contrário, criamos uma lista de nós através das possíveis jogadas disponíveis.
    parent.setChildren(children)                                                                # Atribuímos a lista de nós filhos ao respetivo nó pai.

    return True                                                                                 # Retornamos True indicando que poderá ser feita a expansão.

No caso da *Expansion*, a não ser que L (nó folha) seja um estado final (vitória/empate/derrota) para qualquer jogador, é criada uma lista com os nós com as possíveis jogadas no dado estado do jogo. Após isso é escolhido um dos nós filhos de forma aleatória. Isto porque como os nós acabaram todos de ser criados, o seu valor N (número de visitas durante as simulações) é sempre 0. Segundo a nossa implementação, o valor do nó passa a ser infinito, para que esse nó tenha prioridade sobre outros nós não explorados em futuras pesquisas. Por esse mesmo motivo é indiferente o nó que escolhemos (esta escolha poderia também ser feita por ordem númerica de forma a simplificar, mas por decisão conjunta no trabalho decidimos que seria mais correta esta abordagem para a escolha ser completamente imparcial).

Nota: No código, a escolha do nó é feita ainda na *Selection*, apesar de descrita neste mesmo contexto.

##### **Rollout / Simulation:**

In [ ]:
def __rollOut(self, state):
    while not state.gameOver():                                                                 # Enquanto o jogo não estiver num estado final:
        state.makeMove(random.choice(state.getLegalMoves())+1, self.__rotateSymbol())           # Realizamos jogadas aleatórias, alternando entre os dois possíveis jogadores.
    
    if state.gameDraw():                                                                        # Se o resultado for um empate:
        return ''                                                                               # Retornamos esse mesmo indicador, que irá ser usado na BackPropagation
    
    return state.result                                                                         # Caso não seja um empate, retornamos o vencedor do jogo ('X' ou 'O')


Para os *Rollouts* ou *Simulations*, o código é bastante simples. A partir de um dado estado do jogo, fazemos jogadas alternadas entre os dois jogadores, até chegarmos a um resultado final, devolvendo no final esse mesmo resultado, que será utilizado na *Backpropagation*, para atualizar os valores dos nós. Apenas há um pormenor, quando o resultado é um empate devolvemos um estado diferente de 'X' e 'O', apenas para que na *Backpropagation* este não seja atribuído como vitória para nenhum dos jogadores.



##### **Backpropagation:**

In [ ]:
def __backPropagation(self, node, turn, outcome):
                                                                                                # O turn corresponde ao turno do próximo jogador, daí a seguinte formula de cálculo:
        reward = 0 if outcome == turn else 1                                                    # Se o turno do próximo jogador for o mesmo que o resultado, a reward é 0. Caso seja diferente, a reward é 1.

        while node is not None:                                                                 # Enquanto o nó não for vazio:
            node.N += 1                                                                         # Adicionamos sempre 1 ao N, pois este representa o número de visitas ao nó.
            node.Q += reward                                                                    # Adicionamos a respetiva reward ao valor Q do nó, calculada anteriormente.
            node = node.parent                                                                  # Atualizamos o nó para o seu nó pai.
            if outcome == '':                                                                   # Necessitamos também de verificar se a outcome for um empate, onde não atribuímos pontos a nenhum jogador.
                reward = 0                                              
            else:                                                                               # Caso não seja um empate, alternamos a reward entre 1 e 0.
                reward = 1 - reward

Quanto à *Backpropagation*, esta fica encarregue de subir a árvore e atualizar os valores dos nós a partir do nó onde foi feito o *Rollout*, até à root. Como o jogo funciona por turnos, é necessário atribuir os pontos da vitória apenas quando o nó corresponde a um turno do vencedor. Atribuimos +1 por cada vitória ao valor Q do nó em questão, ou +0 caso o nó não corresponda a um turno do jogador vencedor no respectivo *Rollout*. Por fim, descartamos este *Rollout*.

Desta forma, **com estes 4 métodos, definimos a implementação do MCTS**. No entanto, definimos ainda alguns métodos para legibilidade do código assim como uma correta estruturação.



##### **Search:**

In [ ]:
def search(self, timeLimit):
        startTime = time.process_time()

        numRollouts = 0

        while time.process_time() - startTime < timeLimit:
            node, state = self.__selection()
            outcome = self.rollOut(state)
            self.backPropagation(node, self.symbol, outcome)
            numRollouts += 1

        self.runTime = time.process_time() - startTime
        self.numRollouts = numRollouts

O método *Search* tem como objetivo a junção dos 4 métodos que definem o MCTS (*Selection, Expansion, Rollout, Backpropagation*). É de notar que o Monte Carlo Tree Search tem duas possíveis formas de ser executado:

1 -> Tempo limite

2 -> Número definido de *Rollouts*

No nosso ponto de vista, é mais correto usar o tempo limite. Apesar de sabermos que dependendo da capacidade de processamento de cada computador, o número de *Rollouts* a executar pode variar e por consequência afetar a capacidade de decisão do algoritmo, também é importante a jogabilidade do mesmo. Desta forma, é possível em qualquer máquina, jogar contra o algoritmo em questão, apesar de ser factual que a performance é significativamente alterada.

Também achamos pertinente incluir uma indicação estatística para o número de *Rollouts*, assim como o tempo decorrido. Apesar de ser útil para questões de debug, esta também pode ter influência sobre a capacidade de decisão do algoritmo. Pois quanto mais elaborado for o código, menor quantidade de *Rollouts* conseguirá fazer, influenciando a precisão dos resultados.

##### **Best Move**

In [ ]:
def bestMove(self):

        max_value = max(self.root.children.values(), key=lambda n: n.N).N
        max_nodes = [n for n in self.root.children.values() if n.N == max_value]
        bestChild = random.choice(max_nodes)

        return bestChild

Para determinar a melhor jogada possível, usamos o método *Best Move*. O seu objetivo é apenas manipular os atributos do MCTS e devolver o melhor nó, onde queremos que o MCTS jogue.

Para isso, escolhemos o maior valor de *N*, ou seja, o nó mais explorado, onde obtivemos melhores resultados.

Após isso, comparamos com os restantes valores dos nós filhos da raiz, para obter em caso de empate, todos os nós com os melhores valores. Depois apenas fazemos uma seleção aleatória entre esses mesmos nós com o valor mais interessante, devolvendo no fim um deles.


##### **Notas:**

É importante referir que na classe Node, foi utilizado a UCT (Upper Confidence Bound Applied to Trees). Esta fórmula tem como objetivo equilibrar a diferença entre Exploration e Exploitation quando o algoritmo tem decidir que nós procurar.

A formula é a seguinte:

$$UCT(ni)=\frac{Q}{N}+C{\sqrt\frac{log Np}{N}}$$

Com a mesma foi possível atribuir um valor bem definido a cada nó que posteriormente vem a ser usado na escolha do melhor nó.

##### **Dificuldades na implementação do MCTS:**

Durante a implementação do MCTS, uma das principais dificuldades prendeu-se com a gestão do nó raiz. Inicialmente, não compreendemos que este precisava de ser atualizado após cada jogada - tanto ao ser enviado para a interface como ao ser recebido de volta - o que levou a erros na lógica do algoritmo.

Para resolver este problema, adicionamos à classe *Node* um atributo *children*, implementado como um dicionário onde as chaves representam as colunas jogadas e os valores são os respetivos nós filhos. Esta associação direta entre pai e filhos facilitou bastante a atualização do root após cada jogada. 

Outro desafio surgiu na definição da constante de exploração utilizada no cálculo do valor de cada nó. Após vários testes práticos, encontramos um valor que proporcionava um bom equilíbrio entre exploração de novos nós e o aproveitamento das jogadas já conhecidas como promissoras. Esse valor é **\sqrt{2}**. Valores superiores causavam uma exploração excessiva de caminhos irrelevantes, prejudicando a eficácia do algoritmo.

##### **Análise da relação entre Rollouts/Peças**

Relativamente ao MCTS, ainda foram analisados o número de *Rollouts*, relacionando estes mesmos dados com a quantidade de peças colocadas em jogo até ao momento.

Obtivemos os seguintes dados:


| Valor médio de Rollouts | Peças em Jogo |
| :---------------------: | :-----------: |
| 104250                  | 1             |
| 83000                   | 3             |
| 102250                  | 5             |
| 115000                  | 7             |
| 135600                  | 9             |
| 125000                  | 11            |
| 186000                  | 13            |
| ...                     | ...           |


Colocando estes resultados num gráfico, conseguimos visualmente tirar conclusões.

<div style="text-align: center;">
    <img src="esquemas/grafico.png" alt="Esquema MCTS" width="400" class="center"/>
</div>

Nota: O zig-zag representa uma quebra na escala

Verifica-se que o número de *Rollouts* aumenta significativamente com o aumento do número de peças em jogo. Isto acontece porque à medida que o jogo avança, o número de possíveis estados futuros aumenta exponencialmente, tornando necessário explorar uma maior parte da árvore de pesquisa para avaliar as consequências de cada jogada. Além disso, à medida que o jogo avança, é crucial a análise de posições estratégicas onde há uma vitória iminente. Por esse mesmo motivo, também se entende que as simulações têm de ser mais detalhadas.

##### **Interface**

O código serve para a representação de uma interface que permite a escolha do algoritmo contra quem tencionamos jogar, assim como a escolha do símbolo com que pretendemos jogar.

**Implementação:**


In [ ]:
from Game.searchAlgos.astar import Astar
from Game.searchAlgos.miniMax import MinMax
from Game.searchAlgos.mcts import MCTS
from Game.fourGame import FourGame

# Analisa e retorna a resposta junto com o tabuleiro atual
# params: 
#
#       game | matrix (list of lists): matrix de caracteres | representa o estado atual do tabuleiro do jogo
#       result | type: int | representa o códdigo daquilo que aconteceu no jogo (-1 -> a coluna está cheia | 0 -> Caso não acha fim no jogo | 1 -> Caso de empate | 2 -> Caso de vitória)
#       winner | type: string | representa o símbolo que venceu no caso de vitória
#
# returns: type: boolean | Retorna se o jogo terminou ou não

def showResults(game, result, winner):

    print(game)  # Print do estado atual do tabuleiro

    match result:
        case -1:  # Caso quando a coluna está cheia
            print("Invalid Move! Please choose another column.")
            return False, True
        case 0:  # Caso para quando o movimento é valido mas não resulta no fim do jogo
            print("Nice Move!")
            return False, False
        case 1:  # Caso de empate
            print("It's a Draw!!")
            return True, False
        case 2:  # Caso de vitória
            print('The symbol ' + winner + ' just won!')
            return True, False

def main():
    game = FourGame(7, 6)  # Cria uma nova instância da classe FourGame
    end = False  # Inizializa end como False e usa-o para indicar que o jogo ainda não terminou

    move = input('Escolhe o símbolo com que queres jogar (X ou O): ') # Escolher com que símbolo se quer jogar 

    # Escolhe o símbolo com que o jogador quer jogar
    try:
        if move != 'X' and move != 'O':
            raise ValueError
    except ValueError:
        print('O símbolo deve ser X ou O')
        return
    
    match move:
        case 'X':
            iaSymbol = 'O'
        case 'O': 
            iaSymbol = 'X'

    # Escolhe contra que algoritmo o jogador pretende jogar
    print('===============\n| 1 -> A*     |\n| 2 -> MCTS   |\n| 3 -> MinMax |\n===============')
    algoResp = int(input('Escolhe contra que algoritmo gostarias de jogar: '))
    match algoResp:
        case 1:
            algo = Astar(iaSymbol)
        case 2:
            algo = MCTS(iaSymbol, game)
        case 3:
            algo = MinMax(iaSymbol, move)

    # Faz moves até que o jogo tenha um fim
    while not end:
        invalid = False
        while True:  # Loop para verificar o input até ser dado um movimento válido
            try:
                col = int(input('Column: '))
                if col < 1 or col > 7:
                    raise ValueError  # Raise a ValueError se o input não estiver entre 1-7
                break
            except ValueError:
                print("Invalid Input! Please enter a number from 1 to 7.")

        result, winner = game.makeMove(col, move)  # Faz um movimento na coluna col

        end, invalid = showResults(game, result, winner) # Analisa e retorna a resposta junto com o tabuleiro atual

        if not end and not invalid:
            result, winner = game.makeMove(algo.play(game)+1, iaSymbol)  # Faz um movimento na coluna col

            end, invalid = showResults(game, result, winner) # Analisa e retorna a resposta junto com o tabuleiro atual
        
if __name__ == '__main__':
    main()

##### **Algoritmo VS Algoritmo**

Por motivos de testes e análise de dados, decidimos também fazer um código para os algoritmos jogarem entre si.

**Implementação:**

In [ ]:
from Game.searchAlgos.astar import Astar
from Game.searchAlgos.miniMax import MinMax
from Game.searchAlgos.mcts import MCTS
from Game.fourGame import FourGame

# Analisa e retorna a resposta junto com o tabuleiro atual
# params: 
#
#       game | matrix (list of lists): matrix de caracteres | representa o estado atual do tabuleiro do jogo
#       result | type: int | representa o códdigo daquilo que aconteceu no jogo (-1 -> a coluna está cheia | 0 -> Caso não acha fim no jogo | 1 -> Caso de empate | 2 -> Caso de vitória)
#       winner | type: string | representa o símbolo que venceu no caso de vitória
#
# returns: type: boolean | Retorna se o jogo terminou ou não

def showResults(game, result, winner):

    print(game)  # Print do estado atual do tabuleiro

    match result:
        case -1:  # Caso quando a coluna está cheia
            print("Invalid Move! Please choose another column.")
            return False, True
        case 0:  # Caso para quando o movimento é valido mas não resulta no fim do jogo
            print("Nice Move!")
            return False, False
        case 1:  # Caso de empate
            print("It's a Draw!!")
            return True, False
        case 2:  # Caso de vitória
            print('The symbol ' + winner + ' just won!')
            return True, False

def AstarVsMinMax():
    game = FourGame(7, 6)
    astar = Astar('O')
    minMax = MinMax('X', 'O')
    end = False

    while not end:
        result, winner = game.makeMove(astar.play(game, None)+1, 'O')  # Faz um movimento na coluna col

        end, invalid = showResults(game, result, winner) # Analisa e retorna a resposta junto com o tabuleiro atual

        if not end and not invalid:
            result, winner = game.makeMove(minMax.play(game, None)+1, 'X')  # Faz um movimento na coluna col

            end, invalid = showResults(game, result, winner) # Analisa e retorna a resposta junto com o tabuleiro atual

    
    
def AstarVsMTCS(): 
    game = FourGame(7, 6)
    astar = Astar('O')
    mcts = MCTS('X', game)

    end = False

    while not end:
        move = astar.play(game, None)+1
        result, winner = game.makeMove(move, 'O')  # Faz um movimento na coluna col

        end, invalid = showResults(game, result, winner) # Analisa e retorna a resposta junto com o tabuleiro atual

        if not end and not invalid:
            result, winner = game.makeMove(mcts.play(game, move)+1, 'X')  # Faz um movimento na coluna col

            end, invalid = showResults(game, result, winner) # Analisa e retorna a resposta junto com o tabuleiro atual


def MinMaxVsMTCS(): 
    game = FourGame(7, 6)
    minMax = MinMax('O', 'X')
    mcts = MCTS('X', game)

    end = False

    while not end:
        move = minMax.play(game, None)+1
        result, winner = game.makeMove(move, 'O')  # Faz um movimento na coluna col

        end, invalid = showResults(game, result, winner) # Analisa e retorna a resposta junto com o tabuleiro atual

        if not end and not invalid:
            result, winner = game.makeMove(mcts.play(game, move)+1, 'X')  # Faz um movimento na coluna col

            end, invalid = showResults(game, result, winner) # Analisa e retorna a resposta junto com o tabuleiro atual


def main():

    # Escolhe que algoritmos jogam
    print('=======================\n| 1 -> A* vs MiniMax  |\n| 2 -> A* vs MCTS     |\n| 3 -> MinMax vs MCTS |\n=======================')
    algoResp = int(input('Escolhe os algoritmos para jogarem: '))
    match algoResp:
        case 1:
            AstarVsMinMax()
        case 2:
            AstarVsMTCS()
        case 3:
            MinMaxVsMTCS()
        
if __name__ == '__main__':
    main()

**A\* vs MiniMax**

Nos primeiros testes realizados, o MiniMax venceu sempre o A*. Isto acontece porque o A* funciona de forma semelhante a um algoritmo Greedy - não considera as jogadas do adversário e tenta apenas ganhar o mais depressa possível. Por esse motivo, acaba por ser facilmente derrotado pelo MiniMax, que é uma versão mais completa e estratégica, já que analisa também as possíveis jogadas do adversário até uma certa profundidade na árvore de jogo.

**A\* vs MCTS**

Tal como no confronto com o MiniMax, também nos testes entre o A* e o MCTS, o A* perdeu sempre. O MCTS consegue facilmente bloquear as tentativas de vitória do A* e acaba por vencer, pois ao contrário do A*, tem em conta o comportamento do adversário. Mesmo com um tempo de pesquisa baixo (entre 2 e 4 segundos), o MCTS mostrou ser suficiente para derrotar o A* sem grandes dificuldades.

**MiniMax vs MCTS**

Nos testes entre o MiniMax e o MCTS, notamos que o MCTS ganhou mais vezes quando teve pelo menos 8 segundos para fazer a sua pesquisa. Foram feitos testes com tempos de 2, 4, 6, 8 e 16 segundos. Numa amostra de 20 jogos, o MCTS venceu cerca de 60% das vezes, o Minimax ganhou 30% e os restantes 10% foram empates.

**Conclusão dos confrontos entre algoritmos**

Com base nos testes realizados entre os diferentes algoritmos, concluímos que a ordem de desemprenho foi a seguinte:

1 - MCTS - o mais forte

2 - MiniMax - com boa performance, mas ligeiramente inferior ao MCTS

3 - A* - o mais fraco, por não considerar o adversário

<br></br>

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


<br></br>

## **2. Decision Trees**

A segunda parte deste trabalho dedica-se às Decision Trees, usando o algoritmo ID3.

##### **Iris Dataset**

Decision Tree para conjunto de dados discretizado:

In [1]:
from Dataset import Dataset
from DecisionTree import DecisionTree

irisDecisionTree = DecisionTree(Dataset().readCSV('datasets/', 'iris', True)) # Árvore do dataset (iris.csv)
irisDecisionTree.DFSPrint()

<petallength>
	1.4: Iris-setosa (12)
	1.3: Iris-setosa (7)
	1.5: Iris-setosa (14)
	1.7: Iris-setosa (4)
	1.6: Iris-setosa (7)
	1.1: Iris-setosa (1)
	1.2: Iris-setosa (2)
	1.0: Iris-setosa (1)
	1.9: Iris-setosa (2)
	4.7: Iris-versicolor (5)
	4.5:
		<sepallength>
			6.4: Iris-versicolor (1)
			5.7: Iris-versicolor (1)
			5.6: Iris-versicolor (1)
			6.2: Iris-versicolor (1)
			6.0: Iris-versicolor (2)
			5.4: Iris-versicolor (1)
			4.9: Iris-virginica (1)
	4.9:
		<sepalwidth>
			3.1: Iris-versicolor (1)
			2.5: Iris-versicolor (1)
			2.8: Iris-virginica (1)
			2.7: Iris-virginica (1)
			3.0: Iris-virginica (1)
	4.0: Iris-versicolor (5)
	4.6: Iris-versicolor (3)
	3.3: Iris-versicolor (2)
	3.9: Iris-versicolor (3)
	3.5: Iris-versicolor (2)
	4.2: Iris-versicolor (4)
	3.6: Iris-versicolor (1)
	4.4: Iris-versicolor (4)
	4.1: Iris-versicolor (3)
	4.8:
		<sepallength>
			5.9: Iris-versicolor (1)
			6.8: Iris-versicolor (1)
			6.2: Iris-virginica (1)
			6.0: Iris-virginica (1)
	4.3: Iris-versicol

Decision Tree para conjunto de dados discretizados (para valores numéricos):

In [2]:
from Dataset import Dataset
from DecisionTree import DecisionTree

irisDecisionTreeBinning = DecisionTree(Dataset().readCSV('datasets/', 'iris', True, True, None, 3), True) # Árvore do dataset (iris.csv) com os valores numéricos discretizados
irisDecisionTreeBinning.DFSPrint()

<petalwidth>
	0.10-0.90: Iris-setosa (50)
	0.90-1.70:
		<petallength>
			2.97-4.93: Iris-versicolor (47)
			4.93-6.90:
				<sepallength>
					5.50-6.70:
						<sepalwidth>
							2.00-2.80: Iris-virginica (3)
					6.70-7.90: Iris-virginica (1)
	1.70-2.50:
		<sepalwidth>
			2.80-3.60:
				<petallength>
					2.97-4.93:
						<sepallength>
							5.50-6.70: Iris-virginica (2)
					4.93-6.90:
						<sepallength>
							6.70-7.90: Iris-virginica (15)
							5.50-6.70: Iris-virginica (11)
			2.00-2.80: Iris-virginica (16)
			3.60-4.40: Iris-virginica (2)


##### **GenDataset.py e GenDatasetBig.py** 

**GenDataset.py** 

In [ ]:
import sys
import os
import csv
import random
from copy import deepcopy


game_path = os.path.abspath(os.path.join(os.path.dirname(__file__), 'Game'))
sys.path.append(game_path)

from Game.searchAlgos.mcts import MCTS
from Game.fourGame import FourGame  

#função que gera o dataset
def gerar_dataset_csv(num_amostras=20000, path='datasets/connect4_dataset.csv', simbolo_ia='X'):
    with open(path, mode='w', newline='') as f:
        writer = csv.writer(f)

        #headers
        header = [f"cell_{row}_{col}" for row in range(6) for col in range(7)]
        header.append("piece_count")
        header.append("best_move")
        writer.writerow(header)

        exemplos_gerados = 0
        tentativas = 0

        while exemplos_gerados < num_amostras and tentativas < num_amostras * 10:
            tentativas += 1
            print(f"\n Tentativa #{tentativas}")

            game = FourGame(columns=7, lines=6)
            jogadas_totais = random.randint(6, 41)  # mid to late-game

            #varia o foco posicional: esquerda, centro, direita ou random extra
            pos_focus = random.choice(["left", "center", "right","random"])

            jogador = 'O' if simbolo_ia == 'X' else 'X'

            for i in range(jogadas_totais - 1):
                legal = game.getLegalMoves()
                if not legal or game.gameOver():
                    break

                #filtra jogadas com base no foco posicional
                if pos_focus == "left":
                    legal_focus = [col for col in legal if col <= 2]
                elif pos_focus == "center":
                    legal_focus = [col for col in legal if 2 <= col <= 4]
                elif pos_focus == "right":
                    legal_focus = [col for col in legal if col >= 4]
                #jogadas mais random
                else:
                    legal_focus = [col for col in legal if col <=6]

                if not legal_focus:
                    legal_focus = legal  

                move = random.choice(legal_focus)
                game.makeMove(move + 1, jogador)
                jogador = 'O' if jogador == 'X' else 'X'

            if game.gameOver():
                print(" Jogo terminou antes do MCTS.")
                continue

            mcts = MCTS(iaSymbol=simbolo_ia, game=deepcopy(game))
            mcts.search(0.5)
            best = mcts.bestMove()
            if not best:
                print("MCTS falhou.")
                continue

            best_move = best.move
            game.makeMove(best_move + 1, simbolo_ia)

            # cria entrada no CSV
            estado = []
            piece_count = 0
            for row in game.state:
                for cell in row:
                    if cell == 'X':
                        estado.append(1)
                        piece_count += 1
                    elif cell == 'O':
                        estado.append(2)
                        piece_count += 1
                    else:
                        estado.append(0)

            writer.writerow(estado + [piece_count, best_move])
            exemplos_gerados += 1
            print(f"Exemplo #{exemplos_gerados} gerado. Foco: {pos_focus}")

    print(f"\n{exemplos_gerados} exemplos gerados com sucesso em {path}")

if __name__ == "__main__":
   
    gerar_dataset_csv(num_amostras=20000)

**GenDatasetBig.py**  

In [ ]:
import sys
import os
import csv
import random
from copy import deepcopy

game_path = os.path.abspath(os.path.join(os.path.dirname(__file__), 'Game'))
sys.path.append(game_path)

from Game.searchAlgos.mcts import MCTS
from Game.fourGame import FourGame  

def gerar_dataset_csv(num_amostras=100000, path='datasets/connect4_dataset.csv', simbolo_ia='X'):
    with open(path, mode='w', newline='') as f:
        writer = csv.writer(f)

        #headers
        header = [f"cell_{row}_{col}" for row in range(6) for col in range(7)]
        header += ["piece_count", "first_player", "current_player", "best_move"]
        writer.writerow(header)

        exemplos_gerados = 0
        tentativas = 0

        while exemplos_gerados < num_amostras and tentativas < num_amostras * 10:
            tentativas += 1
            print(f"\nTentativa #{tentativas}")

            game = FourGame(columns=7, lines=6)
            jogadas_totais = random.randint(6, 41)  # de mid a late-game

            # Escolhe aleatoriamente o primeiro jogador
            first_player = random.choice(['X', 'O'])
            jogador = first_player if simbolo_ia == 'O' else ('O' if first_player == 'X' else 'X')

            # Varia o foco posicional: esquerda, centro, direita, aleatório
            pos_focus = random.choice(["left", "center", "right", "random"])

            for i in range(jogadas_totais - 1):
                legal = game.getLegalMoves()
                if not legal or game.gameOver():
                    break

                if pos_focus == "left":
                    legal_focus = [col for col in legal if col <= 2]
                elif pos_focus == "center":
                    legal_focus = [col for col in legal if 2 <= col <= 4]
                elif pos_focus == "right":
                    legal_focus = [col for col in legal if col >= 4]
                else:
                    legal_focus = legal

                if not legal_focus:
                    legal_focus = legal 

                move = random.choice(legal_focus)
                game.makeMove(move + 1, jogador)
                jogador = 'O' if jogador == 'X' else 'X'

            # Última jogada com MCTS 
            if game.gameOver():
                print("Jogo terminou antes do MCTS.")
                continue

            mcts = MCTS(iaSymbol=simbolo_ia, game=deepcopy(game))
            mcts.search(0.5)
            best = mcts.bestMove()
            if not best:
                print("MCTS falhou.")
                continue

            best_move = best.move
            game.makeMove(best_move + 1, simbolo_ia)
            current_player = simbolo_ia

            #cria entrada do CSV
            estado = []
            piece_count = 0
            for row in game.state:
                for cell in row:
                    if cell == 'X':
                        estado.append(1)
                        piece_count += 1
                    elif cell == 'O':
                        estado.append(2)
                        piece_count += 1
                    else:
                        estado.append(0)

            writer.writerow(estado + [piece_count, first_player, current_player, best_move])
            exemplos_gerados += 1
            print(f"Exemplo #{exemplos_gerados} gerado. Foco: {pos_focus}")

    print(f"\n{exemplos_gerados} exemplos gerados com sucesso em {path}")

if __name__ == "__main__":
    gerar_dataset_csv(num_amostras=100000)

Os códigos partilham a mesma estrutura base, a principal distinção está nos atributos utilizados. A explicação detalhada encontra-se na secção "Connect4 Dataset" deste notebook.

Resumo:
    
    -> Define-se aleatoriamente um número de jogadas entre 6 e 41.
    -> Seleciona-se aleatoriamente o jogador inicial, alternando entre ambos a cada jogada.
    -> Determina-se uma preferência de zonas do tabuleiro (direita, esquerda, centro ou distribuição totalmente aleatória).
    -> As jogadas são feitas tendo em conta estas preferências em consideração.
    -> Na jogada final, recorre-se ao algoritmo MCTS para encontrar a melhor jogada possível, tendo em conta o estado atual do tabuleiro.

##### **Estruturas de Dados**

**Node**

In [ ]:
class Node():

    def __init__(self, attribute, value, label, isClass = False, counter = None):
        self.attribute = attribute
        self.isClass = isClass
        self.label = label
        self.value = value
        if (not isClass):
            self.neighbours = []
        else:
            self.counter = counter

    def addNeighbour(self, neighbour):
        self.neighbours.append(neighbour)
    
    def getNeighbours(self):
        return self.neighbours
        
    def getAttribute(self):
        return self.attribute

    def getValue(self):
        return self.value

**Construtor da Classe**

In [ ]:
def __init__(self, attribute, value, label, isClass = False, counter = None):
    self.attribute = attribute
    self.isClass = isClass
    self.label = label
    self.value = value
    if (not isClass):
        self.neighbours = []
    else:
        self.counter = counter

Atributos:

-> 'attribute': Indica o atributo pelo qual este nó realiza a divisão. Quando o campo isClass está definido como True, este valor corresponde à classe final (folha).

-> 'value': Valor associado ao nó em questão.

-> 'isClass': (Opcional, por defeito é False) Booleano que indica se o nó corresponde a uma classe final — ou seja, se é uma folha na árvore.

-> 'label': Nome do atributo que está a ser analisado neste ponto.

-> 'neighbours' - Lista (inicialmente vazia) onde são armazenados os nós-filho, caso este nó não seja uma folha.

-> 'counter' - Número de exemplos do conjunto de dados que resultam no valor representado por este nó, no caso de se tratar de uma classe.<br/><br/>

**Métodos**

Os métodos definidos nesta estrutura de dados limitam-se a devolver (getters) a informação previamente armazenada nos respetivos atributos.

In [ ]:
def getNeighbours(self):
        return self.neighbours
        
def getAttribute(self):
    return self.attribute

def getValue(self):
    return self.value

Existe uma exceção no método 'addNeighbour':

In [ ]:
def addNeighbour(self, neighbour):
        self.neighbours.append(neighbour)

Este método permite efetuar alterações no nó, nomeadamente adicionar nós vizinhos à lista de filhos previamente criada nesse nó.



#### **Dataset**

A próxima classe permite tratar os dados do dataset como uma matriz, de forma a facilitar a sua manipulação e processamento.

In [ ]:
import csv                                                                              
import copy                                                                             

class Dataset():

    def __init__(self, dataset=None, header=None):                                      
        if (dataset != None) and (header != None):                                      
            self.array = dataset                                                        
            self.header = header                                                        
            self.lines = len(self.array)                                                
            self.cols = len(self.array[0])                                              
    
    def readCSV(self, path, filename, hasId=False, hasHeader=True, headerInput=None):   
        csvFile = open(path + filename + '.csv', 'r')                                   
        reader = csv.reader(csvFile)                                                    

        if hasHeader:                                                                   
            self.header = next(reader)                                                  
            self.header.pop(0)                                                          
        else:
            self.header = headerInput                                                   

        self.array = []                                                                 
        for row in reader:                                                              
            self.array.append(row)                                                      

        if hasId:                                                                       
            for i in range(len(self.array)):                                            
                self.array[i].pop(0)                                                    

        self.lines = len(self.array)                                                    
        self.cols = len(self.array[0])                                                  
        return self                                                                     
    
    def getValue(self, line, col):                                                      
        return self.array[line][col]

    def copy(self):                                                                     
        return Dataset(copy.deepcopy(self.array), copy.deepcopy(self.header))
    
    def removeLine(self, line):                                                         
        self.array.pop(line)                                                            
        self.lines -= 1                                                                

    def removeColumn(self, col):                                                        
        if self.header: self.header.pop(col)                                            
        for i in range(len(self.array)):                                                
            self.array[i].pop(col)                                                     
        self.cols -= 1                                                                 

##### **Construtor da Classe**

In [ ]:
def __init__(self, dataset = None, header = None):                                  # Inicializa a classe Dataset com dataset e header opcionais
    if ((dataset != None) and (header != None)):                                    # Verifica se dataset e header não são None
        self.array = dataset                                                        # Atribui dataset ao atributo array da instância
        self.header = header                                                        # Atribui header ao atributo header da instância
        self.lines = len(self.array)                                                # Calcula e armazena o número de linhas do dataset
        self.cols = len(self.array[0])                                              # Calcula e armazena o número de colunas do dataset

##### **Atributos:**

-> 'self.array': Contém os dados do dataset propriamente ditos 

-> 'self.header': Guarda os nomes de cada coluna.

-> 'self.lines': Indica o total de linhas presentes no dataset.

-> 'self.cols': Indica o total de colunas do dataset.

#### **Métodos**

Método 'readCSV':

-> Responsável por ler um ficheiro CSV e carregar os dados para a instância da classe Dataset.

In [ ]:
def readCSV(self, path, filename, hasId=False, hasHeader=True, headerInput = None, binCount = None): # Lê um arquivo CSV e carrega os dados no dataset
        csvFile = open(path+filename+'.csv', 'r')                                   # Abre o arquivo CSV no modo de leitura
        reader = csv.reader(csvFile)                                                # Cria um objeto reader para iterar sobre as linhas do CSV

        if (hasHeader == True):                                                     # Se o arquivo CSV tem um cabeçalho
            self.header = next(reader)                                              # Lê a primeira linha como cabeçalho
            self.header.pop(0)                                                      # Remove o primeiro elemento do cabeçalho (presumivelmente um ID)
        else:
            self.header = headerInput                                               # Se não houver cabeçalho, usa o headerInput fornecido

        self.array = []                                                             # Inicializa o array para armazenar os dados
        for row in reader:                                                          # Itera sobre as linhas restantes do CSV
            self.array.append(row)                                                  # Adiciona cada linha ao array

        if (hasId):                                                                 # Se as linhas têm um ID
            for i in range(len(self.array)):                                        # Itera sobre todas as linhas
                self.array[i].pop(0)                                                # Remove o primeiro elemento de cada linha (presumivelmente um ID)

        self.lines = len(self.array)                                                # Calcula e armazena o número de linhas do array
        self.cols = len(self.array[0])                                              # Calcula e armazena o número de colunas do array

        if (binCount != None):                                                      # Se binCount não for None                
            self.binning(binCount)                                                  # Chama a função binning com o valor de binCount
            
        return self                                                                 # Retorna a instância do dataset

O método 'getValue':

-> Devolve o valor contido numa célula específica do dataset, usando os índices da linha e da coluna indicados.

In [ ]:
def getValue(self, line, col):                                                      # Retorna o valor na linha e coluna especificadas
        return self.array[line][col]

O método 'copy':

-> Cria e devolve uma cópia profunda da instância do Dataset, assegurando que modificações na cópia não influenciem o objeto original.

In [ ]:
def copy(self):                                                                     # Retorna uma cópia profunda do dataset
        return Dataset(copy.deepcopy(self.array), copy.deepcopy(self.header))

O método 'removeLine':

-> Elimina do dataset a linha correspondente ao índice indicado.

In [ ]:
def removeLine(self, line):                                                         # Remove a linha especificada do dataset
        self.array.pop(line)                                                        # Remove a linha do array
        self.lines -= 1                                                             # Decrementa o contador de linhas

O método 'removeColumn':

-> Remove uma coluna do dataset pelo índice fornecido, atualizando também o cabeçalho caso este exista.

In [ ]:
def removeColumn(self, col):                                                        # Remove a coluna especificada do dataset
        if (self.header): self.header.pop(col)                                      # Se houver um cabeçalho, remove a coluna correspondente
        for i in range(len(self.array)):                                            # Itera sobre todas as linhas do array
            self.array[i].pop(col)                                                  # Remove a coluna de cada linha
        self.cols -= 1                                                              # Decrementa o contador de colunas

O método 'binning':

-> Agrupa os valores numéricos em intervalos ("bins"), reduzindo a dimensão do dataset. Esta operação pode, no entanto, implicar uma ligeira perda de informação.

In [ ]:
def binning(self, binCount):                                                        # Bins continuous data into discrete intervals.
    for col in range(self.cols - 1):                                                # Assuming last column is the target
        try:                                                                        # Check if column is numeric
            colData = [float(self.array[i][col]) for i in range(self.lines)]        # Convert column data to float
        except ValueError:                                                          # If column is not numeric, skip
            continue                                                                

        minVal, maxVal = min(colData), max(colData)                                 # Get min and max values of column
        binWidth = (maxVal - minVal) / binCount                                     # Calculate bin width
        bins = [minVal + i * binWidth for i in range(binCount + 1)]                 # Create bins

        for i in range(self.lines):                                                 # Iterate through rows
            value = float(self.array[i][col])                                       # Get value of cell
            for b in range(len(bins) - 1):                                          # Iterate through bins
                if (bins[b] <= value < bins[b + 1]):                                # Check if value is within bin
                    self.array[i][col] = f"{bins[b]:.2f}-{bins[b + 1]:.2f}"         # Replace value with bin range
                    break                                                           # Exit loop
                else:                                                               # If value is not within bin
                    self.array[i][col] = f"{bins[-2]:.2f}-{bins[-1]:.2f}"           # Replace value with last bin range

### **Decision Tree Learning Algorithm: ID3**

O código abaixo define a classe *ID3*, responsável pela construção de uma árvore de decisão com recurso ao *algoritmo ID3* — um dos métodos base na área da classificação de dados e apoio à decisão.

In [ ]:
import numpy as np

class ID3():  

    def __init__(self, dataset):
        self.dataset = dataset
        self.dataSetEntropy = self.__calcDatasetEntorpy()
        self.bestAtributte = self.__getBestGainAtributte()

    # Função de cálculo da entropia de determinado array com valores das classes 
    def __Entropy(self, X):
        sum = 0
        for i in X:
            if (X[i] == 1.0): return 0
            sum += -(X[i]) * np.log2(X[i])
        return sum
    
    # Função de cálculo da entropia do dataset
    def __calcDatasetEntorpy(self):
        values = {}
        for line in range(0, self.dataset.lines):
            value = self.dataset.getValue(line, self.dataset.cols-1)
            if (not (value in values)):
                values[value] = 1
            else:
                values[value] += 1

        for key in values:
            values[key] /= self.dataset.lines

        return self.__Entropy(values)
    
    # Função de decisão e escolha do melhor atributo do dataset apresentado
    def __getBestGainAtributte(self):
        maxGain = float('-inf')
        colMax = 0
        valuesMax = {}
        for j in range(0, self.dataset.cols-1):
            values = {}
            gain = self.dataSetEntropy
            for i in range(0, self.dataset.lines):
                value = self.dataset.getValue(i, j)

                if (not (value in values)):
                    values[value] = {"total": 0}
                
                classVar = self.dataset.getValue(i, self.dataset.cols-1)

                if (not (classVar in values[value])):
                    values[value][classVar] = 1
                else:
                    values[value][classVar] += 1
                
                values[value]["total"] += 1

            for key in values:
                for key2 in values[key]:
                    if key2 != "total":
                        values[key][key2] /= values[key]["total"]
                total = values[key].pop("total")
                gain -= (total/self.dataset.lines) * self.__Entropy(values[key])
                values[key]["total"] = total

            if (gain > maxGain):
                maxGain = gain
                colMax = j
                valuesMax = values

        return colMax, valuesMax

De seguida, será feita uma explicação detalhada da classe e dos seus métodos.<br/><br/>

### **Classe: 'ID3'**

A classe 'ID3' inclui métodos para calcular a entropia de um conjunto de dados, identificar o atributo mais adequado para dividir os dados e dar início à construção da árvore de decisão.<br/><br/>

##### **Construtor da classe**

In [ ]:
def __init__(self, dataset):
    self.dataset = dataset
    self.dataSetEntropy = self.__calcDatasetEntropy()
    self.bestAtributte = self.__getBestGainAttribute()

Atributos:

-> 'dataset' -  Armazena o dataset fornecido (instância da classe Dataset).

-> 'dataSetEntropy' - Guarda a entropia total do dataset, calculada através do método '__calcDatasetEntropy'.

-> 'bestAttribute' - Contém o atributo com maior ganho de informação, determinado pelo método '__getBestGainAttribute'.<br/><br/>

##### **Método: Entropia**

Este método calcula a entropia do dataset, uma medida que quantifica o grau de incerteza ou imprevisibilidade nos dados.



In [ ]:
def __Entropy(self, X):
    sum = 0                                                                     # Inicializamos a soma com 0
    for i in X:                                                                 # Para cada elemento do dicionário X
        if (X[i] == 1.0): return 0                                              # Se o valor do elemento for 1, a entropia é 0
        sum += -(X[i]) * np.log2(X[i])                                          # Caso contrário, a soma fica com o simétrico do valor do elemento multiplicado pelo logaritmo de base 2 do valor do mesmo
    return sum                                                                  # Retornamos a soma


Parâmetros:

- 'X' - Dicionário que representa a distribuição de frequências das diferentes classes.<br/><br/>

Retorno:

- Devolve o valor da entropia correspondente ao dataset.<br/><br/>

**Cálculo da entropia:**

A entropia é determinada com base na fórmula:<br/><br/>

$$H(X) = -\sum p(x) \log_2 p(x)$$

Onde p(x) é a probabilidade de ocorrência da classe x.<br/><br/>

##### **Método: Cálculo da Entropia**

Este método calcula a entropia de todo o dataset. Para isso, começa por determinar a distribuição de frequências das classes e, em seguida, aplica a fórmula apresentada anteriormente para obter o valor da entropia.

In [ ]:
def __calcDatasetEntropy(self):
    values = {}                                                                 # Dicionário para guardar a contagem de ocorrencias de cada classe
    for line in range(0, self.dataset.lines):                                   # Para cada linha do dataset                
        value = self.dataset.getValue(line, self.dataset.cols - 1)              # Obtemos o valor da classe
        if (not (value in values)):
            values[value] = 1                                                   # Se o valor ainda não estiver no dicionário, inicialziamos com o valor 1
        else:
            values[value] += 1                                                  # Caso já esteja no dicionário, incrementamos o valor em 1

    for key in values:                                                          # Para cada par chave-valor no dicionário                                           
        values[key] /= self.dataset.lines                                       # Calculamos a probabilidade de cada classe dividindo o número de ocorrencias pelo número total de linhas

    return self.__Entropy(values)                                               # Calculamos a entropia do dataset

Resumo:

> Inicializa-se um dicionário vazio chamado values, destinado a armazenar a contagem de ocorrências de cada classe.

> Itera-se sobre cada linha do dataset, recolhendo o valor da classe respetiva e atualizando o dicionário values com o número de vezes que cada classe ocorre.

Retorno:

> Devolve a entropia do dataset, recorrendo ao método '__Entropy' descrito anteriormente.

<br/><br/>

##### **Método: Obter o atributo com maior ganho**

Este método identifica qual o atributo (coluna) do dataset que gera o maior ganho de informação. Esse valor é essencial para decidir qual atributo utilizar na divisão dos dados em cada nó da árvore de decisão.

In [ ]:
def __getBestGainAttribute(self):
    maxGain = float('-inf')                                                     # Inicializamos o ganho máximo com o menor valor possível, para garantir que qualquer ganho seja maior
    colMax = 0                                                                  # Inicializamos colmax a 0, que representa o índice da coluna com o maior ganho                         
    valuesMax = {}                                                              # Inicializamos valuesMax como um dicionário vazio, que guardará os valores de cada atributo da coluna com o maior ganho
    for j in range(0, self.dataset.colls - 1):                                  # Para cada coluna do dataset, exceto a última (que contém as classes)
        values = {}                                                             # Inicializamos um dicionário vazio para guardar os valores de cada valor do atributo
        gain = self.dataSetEntropy                                              # Inicializamos o ganho com a entropia do dataset
        for i in range(0, self.dataset.lines):                                  # Para cada linha do dataset
            value = self.dataset.getValue(i, j)                                 # Obtemos o valor do atributo na coluna j

            if (not (value in values)):                                         # Se o valor ainda não estiver no dicionário de valores                                  
                values[value] = {"total": 0}                                    # Inicializamos o valor no dicionário com um dicionário vazio, que guardará a contagem de cada classe

            classVar = self.dataset.getValue(i, self.dataset.colls - 1)         # Obtemos o valor da classe

            if (not (classVar in values[value])):                               # Se a classe ainda não estiver no dicionário do valor do atributo
                values[value][classVar] = 1                                     # Inicializamos a classe no dicionário do valor value com o valor 1
            else:                                                               # Caso contrário
                values[value][classVar] += 1                                    # Incrementamos o valor da classe no dicionário do valor value

            values[value]["total"] += 1                                         # Incrementamos o total de valores do atributo

        for key in values:                                                      # Para cada chave no dicionário de valores                            
            for key2 in values[key]:                                            # Para cada chave no dicionário de valores do atributo                      
                if key2 != "total":                                             # Se a chave não for "total"                            
                    values[key][key2] /= values[key]["total"]                   # Calculamos a probabilidade de cada classe dividindo o número de ocorrências pelo número total de valores do atributo
            total = values[key].pop("total")                                    # Removemos o total do dicionário de valores do atributo
            gain -= (total / self.dataset.lines) * self.__Entropy(values[key])  # Calculamos o ganho subtraindo a entropia do atributo multiplicada pela probabilidade do atributo
            values[key]["total"] = total                                        # Recuperamos o total do dicionário de valores do atributo

        if (gain > maxGain):                                                    # Se o ganho for maior que o ganho máximo
            maxGain = gain                                                      # Atualizamos o ganho máximo
            colMax = j                                                          # Atualizamos o índice da coluna com o maior ganho
            valuesMax = values                                                  # Atualizamos os valores do atributo da coluna com o maior ganho

    return colMax, valuesMax                                                    # Retornamos o índice da coluna com o maior ganho e os valores de cada atributo da coluna

Resumo:

> Percorrem-se todas as colunas (atributos) do dataset.

> Conta-se a frequência de cada par (valor do atributo, classe) e o total de ocorrências de cada valor do atributo para calcular as probabilidades.

> Calcula-se o ganho de informação para cada atributo.

> Atualiza-se o valor máximo de ganho e o índice da melhor coluna sempre que for encontrado um ganho superior.

Retorno:

> Devolve um tuplo (colMax, valuesMax), onde colMax é o índice da coluna com o maior ganho de informação e valuesMax corresponde aos diferentes valores desse atributo e respetivas frequências.

<br/><br/>

### **Decision Tree**

Com as definições do algoritmo ID3 e da estrutura de dados Node concluídas, podemos agora avançar para a implementação da Árvore de Decisão propriamente dita.

In [ ]:
from ID3 import ID3
from Node import Node
from Dataset import Dataset
import copy

class DecisionTree():

    def __init__(self, dataset, binning = False):
        self.initialDataset = dataset
        self.root = self.__generateNode(dataset)
        self.binning = binning

    def __generateNode(self, dataset, tabI=0, numRemovedColumns=0, value=None):
        attribute, values = ID3(dataset).bestAtributte
        node = Node(attribute, value, dataset.header[attribute])

        for key in values:
            isClass = False
            maxValue = float('-inf')
            maxkey = 'failed'
            maxkey2 = 'failed'
            for key2 in values[key]:
                if (key2 != "total"):
                    if (values[key][key2] == 1.0):
                        node.addNeighbour(Node(key2, key, None, True, values[key]["total"]))
                        isClass = True
                        break
                    elif (len(dataset.header) == 2):
                        if maxValue < values[key][key2] and key2 != "total":
                            maxValue = values[key][key2]
                            maxkey = key
                            maxkey2 = key2

            if (not isClass):
                if (len(dataset.header) == 2):
                    node.addNeighbour(Node(maxkey2, maxkey, None, True, int(values[maxkey]["total"]*values[maxkey][maxkey2])))
                else:
                    
                    datasetCopy = dataset.copy()
                    linestoRemove = []
                    for i in range(len(datasetCopy.array)):
                        if (datasetCopy.array[i][attribute] != key):
                            linestoRemove.append(i)

                    for x in sorted(linestoRemove, reverse=True):
                        datasetCopy.removeLine(x)

                    datasetCopy.removeColumn(attribute)

                    node.addNeighbour(self.__generateNode(datasetCopy, tabI+2, numRemovedColumns+1, key))

        return node

    def DFSPrint(self, tabI = 0, node = None):
        
        if (node == None):
            node = self.root
        
        print('\t'*tabI + '<'+node.label+'>')
        
        for currentNode in node.getNeighbours():

            if (currentNode.isClass):
                print(('\t'*(tabI+1))+currentNode.getValue()+': ' + currentNode.getAttribute() + ' (' + str(currentNode.counter) + ')')

            else:
                print(('\t'*(tabI+1))+currentNode.getValue()+':')
                self.DFSPrint(tabI+2, currentNode)

    def classifyMultipleExamples(self, path, file):

        dataset = Dataset().readCSV(path, file, True, False)
        for line in range(dataset.lines):
            classExmp = self.classifyExample(copy.deepcopy(dataset), line)
            if (classExmp == -1):
                print('Not Found!!')
            else:
                print('Line ' + str(line+1) + ' Class: ' + classExmp)
        return
    
    def classifyExample(self, dataset, line):

        actualNode = self.root
        value = dataset.array[line][actualNode.getAttribute()]

        while (actualNode.isClass != True):
            
            neighbours = actualNode.getNeighbours()
            
            found = False
            for node in (neighbours):
                if (self.binning == False):
                    if node.getValue() == value:
                        dataset.removeColumn(actualNode.getAttribute())
                        actualNode = node
                        if (actualNode.isClass != True): value = dataset.array[line][(actualNode.getAttribute())]
                        found = True
                        break
                else:
                    spliting = node.getValue().split('-')
                    if (spliting[0].replace(".", "", 1).isdigit() == True):
                        minVal = float(spliting[0])
                        maxVal = float(spliting[1])
                        value = float(value)
                        if value >= minVal and value <= maxVal:
                            dataset.removeColumn(actualNode.getAttribute())
                            actualNode = node
                            if (actualNode.isClass != True): value = dataset.array[line][(actualNode.getAttribute())]
                            found = True
                            break
                    else:
                        if node.getValue() == value:
                            dataset.removeColumn(actualNode.getAttribute())
                            actualNode = node
                            if (actualNode.isClass != True): value = dataset.array[line][(actualNode.getAttribute())]
                            found = True
                            break

            if (found == False): return -1
    
        return actualNode.getAttribute()

##### **Construtor da classe**

In [ ]:
def __init__(self, dataset, binning = False):
        self.initialDataset = dataset
        self.root = self.__generateNode(dataset)
        self.binning = binning                           

Atributos:

-> 'initialDataset' - Guarda o dataset original (Instância da classe Dataset).

-> 'root' - Armazena o nó raiz da árvore, gerado pela função '__generateNode'.

-> 'binning' - Boolean que representa se foi ou não realizado binning nos valores numéricos (discretizing)

##### **Método: Criar nós**

Este método realiza uma construção recursiva da árvore de decisão, gerando nós para os atributos mais relevantes e nós filhos correspondentes aos seus valores possíveis. O processo continua até que todos os dados estejam completamente classificados como folhas.

In [ ]:
def __generateNode(self, dataset, tabI=0, value=None):                          # Função recursiva para gerar os nós da árvore    
    attribute, values = ID3(dataset).bestAtributte                              # Obtemos o atributo com o maior ganho e seus valores
    node = Node(attribute, value)                                               # Criamos um nó com o atributo e o valor
    for key in values:                                                          # Para cada valor do atributo com o maior ganho
        isClass = False 
        maxValue = float('-inf')
        maxkey = 'failed'
        maxkey2 = 'failed'                                                        # Inicializamos a variável isClass como False
        for key2 in values[key]:                                                # Para cada chave no dicionário de valores do atributo
            if (key2 != "total"):                                               # Se a chave não for "total"
                if (values[key][key2] == 1.0):                                  # Se o valor da chave for 1
                    node.addNeighbour(Node(key2, key, True))                    # Adicionamos um nó com o valor da chave e o valor do atributo como True
                    isClass = True                                              # Atualizamos a variável isClass para True
                    break
                elif (len(dataset.header) == 2):                                # No caso dos valores terem sofrido binning existe uma pequena chance
                    if maxValue < values[key][key2] and key2 != "total":        # de o mesmo intrevalo ter duas classes então será escolhida a   
                        maxValue = values[key][key2]                            # de maior frequência
                        maxkey = key
                        maxkey2 = key2                                                    
        if (not isClass):
            if (len(dataset.header) == 2):
                node.addNeighbour(Node(maxkey2, maxkey, None, True, int(values[maxkey]["total"]*values[maxkey][maxkey2])))  # Se isClass for False  
            else:
                datasetCopy = dataset.copy()                                        # Copiamos o dataset
                linestoRemove = []                                                  # Inicializamos uma lista vazia para guardar as linhas a serem removidas
                for i in range(len(datasetCopy.array)):                             # Para cada linha do dataset
                    if (datasetCopy.array[i][attribute] != key):                    # Se o valor do atributo for diferente do valor do atributo com o maior ganho
                        linestoRemove.append(i)                                     # Adicionamos o índice da linha à lista de linhas a serem removidas

                for x in sorted(linestoRemove, reverse=True):                       # Para cada índice de linha na lista de linhas a serem removidas
                    datasetCopy.removeLine(x)                                       # Removemos a linha do dataset

                datasetCopy.removeCollum(attribute)                                 # Removemos a coluna do dataset

                node.addNeighbour(self.__generateNode(datasetCopy, tabI+2, key))    # Adicionamos um nó filho gerado recursivamente com o dataset cortado

    return node                                                                     # Retornamos o nó

Resumo:

> Seleciona-se o melhor atributo para dividir os dados, aplicando o algoritmo ID3.

> Cria-se um nó com esse atributo, que se torna a raiz da subárvore atual.

> Para cada valor possível desse atributo, são gerados nós filhos ou folhas.

> Os nós filhos representam as ramificações da árvore, baseadas nos próximos melhores atributos para divisão.


Retorno:

> O método retorna o nó raiz da árvore de decisão, que inclui recursivamente todos os nós filhos correspondentes aos valores do melhor atributo encontrado no conjunto de dados.

<br/><br/>

##### **Método: Imprimir segundo um DFS**

Este método realiza a impressão hierárquica da árvore de decisão, utilizando uma busca em profundidade (DFS) para percorrer todos os seus nós.

In [ ]:
def DFSPrint(self, tabI=0, node=None):                                                             # Função para imprimir a árvore em profundidade
    
    if node is None:                                                                               # Função para imprimir a árvore em profundidade
        node = self.root                                                                           # O nó é a raiz da árvore

    print('\t' * tabI + '<' + str(node.label) + '>')                                               # Imprimimos o nome do atributo do nó
    
    for currentNode in node.getNeighbours():                                                        # Para cada nó vizinho do nó passado como parâmetro
        if currentNode.isClass:                                                                     # Se o nó for uma classe
            print('\t' * (tabI + 1) + str(currentNode.getValue()) + ': ' +                          # Imprimimos o valor do nó
                str(currentNode.getAttribute()) + ' (' + str(currentNode.counter) + ')')                
        else:                                                                                       # Caso contrário
            print('\t' * (tabI + 1) + str(currentNode.getValue()) + ':')                            # Imprimimos o valor do nó
            self.DFSPrint(tabI + 2, currentNode)                                                    # Chamamos a função recursivamente para imprimir os nós filhos

> Facilita a compreensão da estrutura da árvore de decisão.

> Permite uma visualização clara dos atributos e valores em cada nível da árvore.

### **Interface**

**Implementação:**

In [ ]:
from Dataset import Dataset
from DecisionTree import DecisionTree
from copy import deepcopy
import random

irisTree = DecisionTree(Dataset().readCSV('datasets/', 'iris', hasId=True))
irisTreeBinning = DecisionTree(Dataset().readCSV('datasets/', 'iris', hasId=True, binCount=3), binning=True)
connect4Tree = DecisionTree(Dataset().readCSV('datasets/', 'connect4_dataset', hasId=False, hasHeader=True))

def chooseTree():
    print("\nEscolhe uma das árvores:")
    print("[1] Iris")
    print("[2] Iris com Binning")
    print("[3] Connect 4")
    option = input("Opção: ")

    if option == '1':
        return irisTree, 'datasets/', 'iris', True
    elif option == '2':
        return irisTreeBinning, 'datasets/', 'iris', True
    elif option == '3':
        return connect4Tree, 'datasets/', 'connect4_dataset', False
    else:
        print("Opção inválida.")
        return chooseTree()

def main():
    while True:
        print("\nÁrvores de Decisão - Algoritmo ID3")
        print("1. Ver árvore")
        print("2. Classificar ficheiro CSV")
        print("3. Calcular precisão")
        print("4. Sair")

        op = input("Escolha a opção: ")

        if op == '1':
            tree, *_ = chooseTree()
            print("\nÁrvore:")
            tree.DFSPrint()

        elif op == '2':
            tree, path, file, hasId = chooseTree()
            dataset = Dataset().readCSV(path, file, hasId=hasId, hasHeader=True)

            print("\nClassificações:")
            for i in range(dataset.lines):
                predicted = tree.classifyExample(deepcopy(dataset), i)
                print(f"Linha {i + 1}: Classe prevista = {predicted}")

        elif op == '3':
            tree, path, file, hasId = chooseTree()
            dataset = Dataset().readCSV(path, file, hasId=hasId, hasHeader=True)
            random.shuffle(dataset.array)

            split_point = int(0.8 * dataset.lines)
            train_data = Dataset(deepcopy(dataset.array[:split_point]), deepcopy(dataset.header))
            test_data = Dataset(deepcopy(dataset.array[split_point:]), deepcopy(dataset.header))

            tree = DecisionTree(train_data)

            acertos = 0
            total = test_data.lines

            for i in range(total):
                previsto = tree.classifyExample(deepcopy(test_data), i)
                real = test_data.getValue(i, test_data.cols - 1)
                if previsto == real:
                    acertos += 1

            print(f"\nPrecisão no conjunto de teste: {acertos}/{total} = {acertos / total * 100:.2f}%")

        elif op == '4':
            print("Até à próxima!")
            break
        else:
            print("Opção inválida.")

if __name__ == "__main__":
    main()

#### **Connect4 Dataset**

O dataset criado para o problema do Connect4 tem como objetivo representar o estado atual do tabuleiro e prever a melhor jogada seguinte, determinada por simulações do algoritmo Monte Carlo Tree Search (MCTS).

Considerando a complexidade extrema do jogo — com mais de 4,5 triliões de possíveis estados — foram adicionados três atributos complementares para enriquecer a representação do estado:

- **first_player**: identifica quem iniciou a partida (X ou O).

- **current_player**: indica quem está prestes a jogar (responsável pela jogada recomendada pelo MCTS).

- **piece_count**: representa o total de peças já colocadas no tabuleiro até o momento.

A inclusão desses atributos visa diminuir a incerteza na previsão da jogada ideal, ao contextualizar melhor o estado atual do jogo. Por exemplo, dois tabuleiros visualmente idênticos podem representar situações estratégicas distintas dependendo de quem começou a partida ou quem joga na vez atual.

É importante destacar que a utilidade desses atributos depende da quantidade e diversidade dos dados:

- Em conjuntos de dados pequenos, eles podem levar a overfitting, fazendo com que o modelo aprenda padrões muito específicos ou ruidosos.

- Em datasets amplos e equilibrados, esses atributos permitem uma segmentação mais precisa dos estados, ajudando a árvore de decisão a tomar decisões mais generalizáveis e informadas.

Portanto, o uso desses atributos é especialmente indicado em cenários com volumes consideráveis de dados, ajudando a mitigar o impacto do desbalanceamento inerente a um jogo com um espaço de estados tão vasto.

<br/><br/>

#### **Binning e valores numéricos nos Datasets**

Valores numéricos tendem a aumentar significativamente a dimensão da árvore de decisão. No entanto, esses valores podem ser facilmente agrupados em intervalos, técnica que reduz tanto o tamanho do dataset quanto da árvore.

Uma das abordagens para isso é o binning, que consiste em dividir um conjunto de dados numéricos em intervalos (bins). Essa técnica diminui a complexidade dos dados, embora possa causar perda de informação, especialmente quando os intervalos são muito amplos.

O desafio é encontrar um equilíbrio ideal que permita reduzir o tamanho dos dados sem comprometer demasiadamente a precisão do modelo.

Esse algoritmo foi aplicado aos datasets weather e iris, tendo apresentado algumas imprecisões no caso do iris durante os testes.

Esse problema ocorre porque um mesmo intervalo pode conter várias classes diferentes. Para contornar isso, implementou-se uma estratégia que escolhe a classe de maior frequência dentro do intervalo, reduzindo assim os erros.

Por exemplo, se um intervalo contém 3 valores da classe Iris-virginica e 1 valor da classe Iris-versicolor, isso implica que, ao agrupar os quatro valores, um deles (o Iris-versicolor) terá uma classificação incorreta.

![Teste](esquemas/intervalobinning.png "Title")

<br/><br/>

#### **Balanced Datasets**

Existem dois tipos principais de datasets: balanceados e não balanceados.

- **Datasets balanceados** são aqueles em que a frequência das classes é aproximadamente igual entre si.

- **Datasets não balanceados** apresentam diferenças significativas na frequência das classes.

Datasets balanceados tendem a gerar árvores de decisão mais equilibradas. Para alcançar esse equilíbrio, existem duas técnicas comuns de balanceamento:

- **Undersampling**: reduz o número de exemplos das classes mais frequentes até que todas as classes fiquem balanceadas.

- **Oversampling**: aumenta o número de exemplos das classes menos frequentes para alcançar o balanceamento.

No nosso caso, o dataset iris está balanceado, pois cada classe representa aproximadamente 33% dos exemplos, com uma distribuição de 50/50/50 amostras.